# ECH-Workers 编译工具

本笔记本用于在 Google Colab 上编译：
1. **内核 (ech-workers-core.exe)** - Go 语言编写的代理内核
2. **GUI (EchWorkersGui.exe)** - C# WPF 图形界面

**支持平台**：
- 内核：Windows/Linux/macOS (amd64/arm64)
- GUI：仅 Windows (需要 .NET 8)

## 1. 安装编译环境

### 1.1 安装 Go 1.24+ (用于编译内核)

In [ ]:
%%bash
# 下载并安装 Go 1.24
GO_VERSION="1.24.0"
wget -q https://go.dev/dl/go${GO_VERSION}.linux-amd64.tar.gz
sudo rm -rf /usr/local/go
sudo tar -C /usr/local -xzf go${GO_VERSION}.linux-amd64.tar.gz
rm go${GO_VERSION}.linux-amd64.tar.gz

# 验证安装
echo "Go 安装完成:"
/usr/local/go/bin/go version

In [ ]:
# 设置 Go 环境变量
import os
os.environ['PATH'] = '/usr/local/go/bin:' + os.environ['PATH']
os.environ['GOPATH'] = '/root/go'

!go version

### 1.2 安装 .NET 8 SDK (用于编译 GUI)

In [ ]:
%%bash
# 安装 .NET 8 SDK
wget https://packages.microsoft.com/config/ubuntu/22.04/packages-microsoft-prod.deb -O packages-microsoft-prod.deb
sudo dpkg -i packages-microsoft-prod.deb
rm packages-microsoft-prod.deb

sudo apt-get update
sudo apt-get install -y dotnet-sdk-8.0

echo ""
echo ".NET SDK 安装完成:"
dotnet --version

## 2. 上传源代码

将 `ech-workers` 文件夹打包成 zip 上传，或者从 Git 仓库克隆。

In [ ]:
# 方式一：上传 zip 文件
from google.colab import files
import zipfile
import os

print("请上传 ech-workers.zip 文件...")
uploaded = files.upload()

# 解压
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/')
        print(f"已解压: {filename}")
        os.remove(filename)

!ls -la /content/

In [ ]:
# 方式二：从 Git 克隆（如果有仓库的话）
# !git clone https://github.com/your-repo/ech-workers.git /content/ech-workers

## 3. 编译内核 (Go)

In [ ]:
%%bash
cd /content/ech-workers

# 下载依赖
echo "=== 下载依赖 ==="
/usr/local/go/bin/go mod tidy

echo ""
echo "=== 依赖下载完成 ==="

In [ ]:
%%bash
cd /content/ech-workers
mkdir -p /content/build

echo "=== 编译内核 Windows amd64 ==="
GOOS=windows GOARCH=amd64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/ech-workers-core-windows-amd64.exe .

echo "=== 编译内核 Windows arm64 ==="
GOOS=windows GOARCH=arm64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/ech-workers-core-windows-arm64.exe .

echo "=== 编译内核 Linux amd64 ==="
GOOS=linux GOARCH=amd64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/ech-workers-core-linux-amd64 .

echo "=== 编译内核 Linux arm64 ==="
GOOS=linux GOARCH=arm64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/ech-workers-core-linux-arm64 .

echo "=== 编译内核 macOS amd64 ==="
GOOS=darwin GOARCH=amd64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/ech-workers-core-darwin-amd64 .

echo "=== 编译内核 macOS arm64 (Apple Silicon) ==="
GOOS=darwin GOARCH=arm64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/ech-workers-core-darwin-arm64 .

echo ""
echo "=== 内核编译完成 ==="
ls -lh /content/build/ech-workers-core-*

## 4. 编译 GUI (C# WPF)

> **注意**：GUI 仅支持 Windows 平台，需要 .NET 8 运行时

In [ ]:
%%bash
cd /content/ech-workers/gui/EchWorkersGui

echo "=== 编译 GUI Windows x64 (独立 exe) ==="
dotnet publish -c Release -r win-x64 --self-contained true \
  -p:PublishSingleFile=true \
  -p:IncludeNativeLibrariesForSelfExtract=true \
  -p:EnableWindowsTargeting=true \
  -o /content/build/gui-win-x64

echo ""
echo "=== 编译 GUI Windows arm64 (独立 exe) ==="
dotnet publish -c Release -r win-arm64 --self-contained true \
  -p:PublishSingleFile=true \
  -p:IncludeNativeLibrariesForSelfExtract=true \
  -p:EnableWindowsTargeting=true \
  -o /content/build/gui-win-arm64

echo ""
echo "=== GUI 编译完成 ==="
ls -lh /content/build/gui-win-x64/EchWorkersGui.exe
ls -lh /content/build/gui-win-arm64/EchWorkersGui.exe

In [ ]:
## 5. 下载编译产物

%%bash
# 整理编译产物
mkdir -p /content/build/final

# 复制内核
cp /content/build/ech-workers-core-windows-amd64.exe /content/build/final/
cp /content/build/ech-workers-core-windows-arm64.exe /content/build/final/

# 复制 GUI
cp /content/build/gui-win-x64/EchWorkersGui.exe /content/build/final/EchWorkersGui-x64.exe
cp /content/build/gui-win-arm64/EchWorkersGui.exe /content/build/final/EchWorkersGui-arm64.exe

# 打包
cd /content/build/final
zip -r /content/ech-workers-windows.zip .

echo "=== 最终产物 ==="
ls -lh /content/build/final/
echo ""
echo "打包文件: /content/ech-workers-windows.zip"
ls -lh /content/ech-workers-windows.zip

In [ ]:
# 下载 Windows 完整包（内核 + GUI）
from google.colab import files
files.download('/content/ech-workers-windows.zip')

In [ ]:
# 或者单独下载各个文件
from google.colab import files

# 下载内核
# files.download('/content/build/final/ech-workers-core-windows-amd64.exe')

# 下载 GUI
# files.download('/content/build/final/EchWorkersGui-x64.exe')

## 6. 快速编译（仅 Windows x64）

In [ ]:
%%bash
cd /content/ech-workers
mkdir -p /content/build/quick

echo "=== 快速编译内核 Windows x64 ==="
GOOS=windows GOARCH=amd64 /usr/local/go/bin/go build -ldflags="-s -w" -o /content/build/quick/ech-workers-core.exe .

echo "=== 快速编译 GUI Windows x64 ==="
cd /content/ech-workers/gui/EchWorkersGui
dotnet publish -c Release -r win-x64 --self-contained true \
  -p:PublishSingleFile=true \
  -p:IncludeNativeLibrariesForSelfExtract=true \
  -p:EnableWindowsTargeting=true \
  -o /content/build/quick

echo ""
echo "=== 快速编译完成 ==="
ls -lh /content/build/quick/*.exe

In [ ]:
# 下载快速编译产物
from google.colab import files

# 打包
!cd /content/build/quick && zip -r /content/ech-workers-quick.zip *.exe

files.download('/content/ech-workers-quick.zip')